In [3]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import contractions
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob
import re
import pandas as pd
glassdoor_reviews = pd.read_csv("~/Documents/CompCulture/Collabera/Data/reviews_new_processed.csv")
tokenizer = TweetTokenizer()
stop_words = STOP_WORDS
custom_stop_words = ['people', 'collabera', 'employee', 'employees', 'collabera\'s', 'work', 'working', 'company', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa']
#stop_words = set(list(stop_words) + custom_stop_words)
lemmatizer = WordNetLemmatizer()
# getting rid of these tokens that are not included in the pre-defined punctuation list
# no better lists existing
punctuation = string.punctuation + '–...…’“”'
# removing bullet points and numbers
re_number = r"[0-9]+(\.)?"

glassdoor_reviews = glassdoor_reviews.dropna(subset=['pros', 'cons']).astype({'pros':'str', 'cons':'str'})

/Users/Lara/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
glassdoor_reviews = glassdoor_reviews.head(1000)
pros = glassdoor_reviews['pros'].apply(contractions.fix).str.lower().apply(lambda x: x.replace('.',' ')).apply(tokenizer.tokenize).apply(
    lambda toks : [lemmatizer.lemmatize(t) for t in toks
                   if t not in punctuation and t not in stop_words and re.match(re_number, t) is None]).apply(' '.join).to_list()
cons = glassdoor_reviews['cons'].apply(contractions.fix).str.lower().apply(lambda x: x.replace('.',' ')).apply(tokenizer.tokenize).apply(
    lambda toks : [lemmatizer.lemmatize(t) for t in toks
                   if t not in punctuation and t not in stop_words and re.match(re_number, t) is None]).apply(' '.join).to_list()

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

texts = pros
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(texts).toarray()

In [16]:
from biterm.utility import vec_to_biterms
import numpy as np

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)

In [19]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab)
topics = btm.fit_transform(biterms, iterations=100)

100%|██████████| 100/100 [2:07:46<00:00, 76.66s/it] 
/Users/Lara/anaconda3/lib/python3.7/site-packages/biterm/btm.py:76: RuntimeWarning: invalid value encountered in true_divide
  P_zd[i] = P_zb.sum(axis=0) / P_zb.sum(axis=0).sum()


In [ ]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab)
topics = btm.fit_transform(biterms, iterations=100)